In [239]:
import pandas as pd
import numpy as np
from math import sqrt

In [240]:
ends = pd.read_csv('/Users/brentkong/Documents/curling/data/Ends.csv')
games = pd.read_csv('/Users/brentkong/Documents/curling/data/Games.csv')
stones = pd.read_csv('/Users/brentkong/Documents/curling/data/Stones.csv')

In [241]:
stones["GameUID"] = stones["CompetitionID"].astype(str) + "_" + \
                    stones["SessionID"].astype(str) + "_" + \
                    stones["GameID"].astype(str)

ends["GameUID"]   = ends["CompetitionID"].astype(str) + "_" + \
                    ends["SessionID"].astype(str) + "_" + \
                    ends["GameID"].astype(str)

ends["TeamEndUID"]   = ends["CompetitionID"].astype(str) + "_" + \
                    ends["SessionID"].astype(str) + "_" + \
                    ends["GameID"].astype(str) + "_" + \
                    ends["TeamID"].astype(str) + "_" + \
                    ends["EndID"].astype(str)

games["GameUID"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str)

games["TeamUID_1"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str) + "_" + \
                    games["TeamID1"].astype(str)

games["TeamUID_2"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str) + "_" + \
                    games["TeamID2"].astype(str)




In [242]:
games

,CompetitionID,SessionID,GameID,GroupID,Sheet,NOC1,NOC2,ResultStr1,ResultStr2,LSFE,Winner,TeamID1,TeamID2,GameUID,TeamUID_1,TeamUID_2
0,0,1,1,0,A,SWE,GBR,5,9,0,0,19,27,0_1_1,0_1_1_19,0_1_1_27
1,0,1,2,0,B,AUS,USA,5,6,1,0,46,20,0_1_2,0_1_2_46,0_1_2_20
2,0,1,3,0,C,NOR,CZE,6,7,0,0,17,22,0_1_3,0_1_3_17,0_1_3_22
3,0,1,4,0,D,CHN,SUI,7,6,1,1,43,18,0_1_4,0_1_4_43,0_1_4_18
4,0,2,1,0,A,AUS,CHN,5,6,0,0,46,43,0_2_1,0_2_1_46,0_2_1_43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,24250026,46,1,0,B,ITA,EST,7,6,1,1,24,37,24250026_46_1,24250026_46_1_24,24250026_46_1_37
340,24250026,46,2,0,D,AUS,SCO,6,9,1,0,46,14,24250026_46_2,24250026_46_2_46,24250026_46_2_14
341,24250026,46,3,0,C,CAN,USA,4,8,1,0,10,20,24250026_46_3,24250026_46_3_10,24250026_46_3_20
342,24250026,48,1,0,C,AUS,EST,9,2,1,1,46,37,24250026_48_1,24250026_48_1_46,24250026_48_1_37


In [ ]:
def add_hammer_ends():
    map = {}

    for i, row in games.iterrows():
        team_1_id = row['TeamUID_1']
        team_2_id = row['TeamUID_2']
        
        hammer_first_team = team_2_id
        
        game_ends = ends[ends['GameUID'] == row['GameUID']].sort_values('EndID')
        
        hammer = hammer_first_team
        for _, end_row in game_ends.iterrows():
            end_id = end_row['EndID']
            
            map[f"{team_1_id}_{end_id}"] = (hammer == team_1_id)
            map[f"{team_2_id}_{end_id}"] = (hammer == team_2_id)
            
            hammer = team_1_id if hammer == team_2_id else team_2_id

    ends['Hammer'] = ends['TeamEndUID'].map(map)



def add_cumulative_ends():
    map = {}
    
    ends['TeamGameID'] = ends.apply(
        lambda row: f"{row['CompetitionID']}_{row['SessionID']}_{row['GameID']}_{row['TeamID']}",
        axis=1
    )
    
    for team_game_id in ends['TeamGameID'].unique():
        team_ends = ends[ends['TeamGameID'] == team_game_id].sort_values('EndID')
        
        cumulative = 0
        for idx, row in team_ends.iterrows():
            cumulative += row['Result']
            map[row['TeamEndUID']] = cumulative
    
    ends['Cumulative'] = ends['TeamEndUID'].map(map)
    
    ends.drop('TeamGameID', axis=1, inplace=True)

add_hammer_ends()
add_cumulative_ends()

In [270]:
ends.iloc[4321,:]

CompetitionID             24250026
SessionID                        8
GameID                           3
TeamID                          37
EndID                            1
Result                           1
PowerPlay                      NaN
GameUID               24250026_8_3
TeamEndUID       24250026_8_3_37_1
Hammer                        True
Cumulative                       1
Name: 4321, dtype: object

In [245]:
def add_hammer_ends():
    map = {}
    for i, uid in enumerate(games['GameUID'].tolist()):
        team_1_id = games["TeamUID_1"].loc[i]
        team_2_id = games["TeamUID_2"].loc[i]
        parity = (games.iloc[i, 9] % 2 == 0)
        for j in range(1, 9):
            team_game_1_id = team_1_id + "_" + str(j)
            team_game_2_id = team_2_id + "_" + str(j)
            map[team_game_1_id] = (not parity)
            map[team_game_2_id] = parity
            parity = not parity

    ends['Hammer'] = ends['TeamEndUID'].map(map)


In [246]:
ends.head(200)

,CompetitionID,SessionID,GameID,TeamID,EndID,Result,PowerPlay,GameUID,TeamEndUID,Hammer,Cumulative
0,0,1,1,19,1,0,NaN,0_1_1,0_1_1_19_1,True,0
1,0,1,1,27,1,1,NaN,0_1_1,0_1_1_27_1,False,1
2,0,1,1,19,2,2,NaN,0_1_1,0_1_1_19_2,True,2
3,0,1,1,27,2,0,NaN,0_1_1,0_1_1_27_2,False,1
4,0,1,1,19,3,0,NaN,0_1_1,0_1_1_19_3,True,2
...,...,...,...,...,...,...,...,...,...,...,...
195,0,4,2,27,7,3,NaN,0_4_2,0_4_2_27_7,False,7
196,0,4,2,18,8,1,NaN,0_4_2,0_4_2_18_8,True,8
197,0,4,2,27,8,0,NaN,0_4_2,0_4_2_27_8,False,7
198,0,4,3,19,1,1,NaN,0_4_3,0_4_3_19_1,False,1


In [247]:
def add_cumulative_ends():
    map = {}
    uids = [games["TeamUID_1"].tolist(), games["TeamUID_2"].tolist()]
    for team_list in uids:
        for uid in team_list:
            cumulative = 0
            for i in range(1, 9):
                id = str(uid) + "_" + str(i)
                print(id)
                cumulative += ends.loc[ends['TeamEndUID'] == id, 'Result'].iloc[0]
                map[id] = cumulative
        
    ends['Cumulative'] = ends['TeamEndUID'].map(map)


#add_cumulative_ends()
#ends

In [248]:
games

,CompetitionID,SessionID,GameID,GroupID,Sheet,NOC1,NOC2,ResultStr1,ResultStr2,LSFE,Winner,TeamID1,TeamID2,GameUID,TeamUID_1,TeamUID_2
0,0,1,1,0,A,SWE,GBR,5,9,0,0,19,27,0_1_1,0_1_1_19,0_1_1_27
1,0,1,2,0,B,AUS,USA,5,6,1,0,46,20,0_1_2,0_1_2_46,0_1_2_20
2,0,1,3,0,C,NOR,CZE,6,7,0,0,17,22,0_1_3,0_1_3_17,0_1_3_22
3,0,1,4,0,D,CHN,SUI,7,6,1,1,43,18,0_1_4,0_1_4_43,0_1_4_18
4,0,2,1,0,A,AUS,CHN,5,6,0,0,46,43,0_2_1,0_2_1_46,0_2_1_43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,24250026,46,1,0,B,ITA,EST,7,6,1,1,24,37,24250026_46_1,24250026_46_1_24,24250026_46_1_37
340,24250026,46,2,0,D,AUS,SCO,6,9,1,0,46,14,24250026_46_2,24250026_46_2_46,24250026_46_2_14
341,24250026,46,3,0,C,CAN,USA,4,8,1,0,10,20,24250026_46_3,24250026_46_3_10,24250026_46_3_20
342,24250026,48,1,0,C,AUS,EST,9,2,1,1,46,37,24250026_48_1,24250026_48_1_46,24250026_48_1_37


In [249]:
print(not True)

False


### Stone Geometry Features

In [250]:
stones.columns

Index(['CompetitionID', 'SessionID', 'GameID', 'EndID', 'ShotID', 'TeamID',
       'PlayerID', 'Task', 'Handle', 'Points', 'TimeOut', 'stone_1_x',
       'stone_1_y', 'stone_2_x', 'stone_2_y', 'stone_3_x', 'stone_3_y',
       'stone_4_x', 'stone_4_y', 'stone_5_x', 'stone_5_y', 'stone_6_x',
       'stone_6_y', 'stone_7_x', 'stone_7_y', 'stone_8_x', 'stone_8_y',
       'stone_9_x', 'stone_9_y', 'stone_10_x', 'stone_10_y', 'stone_11_x',
       'stone_11_y', 'stone_12_x', 'stone_12_y', 'GameUID'],
      dtype='object')

In [251]:
def dist_to_button(x, y):
    return sqrt((x - 750)**2 + (y - 800)**2)

def ideal_guard_error(x, side):
    ideal = 1200 if side == 1 else 300
    return abs(x - ideal)

def open_lane_proxy(df, side):
    corridor = 1200 if side == 1 else 300
    others = df[~df["Stone_1_x"].isin([0, 4095])]
    if others.empty:
        return 750
    return others["Stone_1_x"].apply(lambda x: abs(x - corridor)).min()

In [252]:
stones.iloc[:, 11:34]

,stone_1_x,stone_1_y,stone_2_x,stone_2_y,stone_3_x,stone_3_y,stone_4_x,stone_4_y,stone_5_x,stone_5_y,...,stone_7_y,stone_8_x,stone_8_y,stone_9_x,stone_9_y,stone_10_x,stone_10_y,stone_11_x,stone_11_y,stone_12_x
0,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,...,650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,750.0,1916.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,963.0,1657.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0
4,963.0,1657.0,4095.0,4095.0,694.0,756.0,636.0,1080.0,0.0,0.0,...,650.0,4095.0,0.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26365,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,0.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,0.0,0.0,0.0
26366,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,760.0,1152.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,0.0,0.0,0.0
26367,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,4095.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,272.0,797.0,0.0
26368,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,4095.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,272.0,797.0,0.0
